<a href="https://colab.research.google.com/github/Keeron1/com.mcast.research_design_I_2025.spiteri_keeron/blob/main/src/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Packages

In [ ]:
# YOLO
!pip install torch torchvision opencv-python ultralytics
# !rm -rf deep_sort_pytorch
# DeepSORT
!git clone https://github.com/Keeron1/deep_sort_pytorch.git # temp
!pip install -r deep_sort_pytorch/requirements.txt

# 2. Import Libraries

In [14]:
from ultralytics import YOLO # Importing YOLO
import cv2 # opencv
from google.colab.patches import cv2_imshow # cv2.imshow("title", frane) doesn't work in Colab so this is the fix cv2_imshow(frame)
import glob
import json
import os

# 3. Config

In [ ]:
dataset_path = "/content/dataset"
dataset_data_path = dataset_path + "/data.yaml"
idx_to_class_path = "/content/idx_to_class.json"

# Load object detector model
# model = YOLO("yolov8m.pt")
model = YOLO("/content/best.pt")
model.info() # Display model information

# DeepSORT cfg
deep_sort_cfg = {
    "USE_FASTREID" : False,
    'DEEPSORT': {
        'REID_CKPT': 'deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7',
        'MAX_DIST': 0.2, # Maximum cosine distance
        'MIN_CONFIDENCE': 0.5, # Minimum detector confidence
        'NMS_MAX_OVERLAP': 0.5, # Maximum IoU for suppressing overlapping boxes.
        'MAX_IOU_DISTANCE': 0.5, # 0.5 Max IoU distance for motion-only matching in the cascade.
        'MAX_AGE': 70, # Number of consecutive frames a track is kept “alive” without matching.
        'N_INIT': 3, # Number of frames a track must be consistently matched before confirmed.
        'NN_BUDGET': 100 # Maximum number of appearance features to store per track.
    }
}


# 4. Download Dataset

## 4.1 Download

## 4.2 Unzip

In [ ]:
!unzip -q /content/pedestrain-data.zip -d {dataset_path}

# 5. Train Model

In [ ]:
# Train the model
results = model.train(data=dataset_data_path, epochs=50, imgsz=640)

Ultralytics 8.3.114 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=

100%|██████████| 755k/755k [00:00<00:00, 24.3MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576, 192, 2]                 
 16                  -1  1    332160  ultralytics.nn.modules.conv.Conv             [192,

100%|██████████| 5.35M/5.35M [00:00<00:00, 69.9MB/s]


...

      50/50      6.38G     0.4317     0.2446     0.8658         47        640: 100%|██████████| 50/50 [00:26<00:00,  1.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.90it/s]

                   all         72        712      0.987       0.99      0.995      0.842



50 epochs completed in 0.390 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.114 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.60it/s]


                   all         72        712      0.987       0.99      0.995      0.845
Speed: 0.3ms preprocess, 10.5ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/train


# 6. Test Model

In [ ]:
# Test the model
# model = YOLO("runs/detect/train/weights/best.pt")

# Run inference with the YOLO model on the validation dataset
results = model.predict(source="people.mp4", save=True, verbose=True) # dataset_path + "/valid/images"

## Display Results

### From predict function

In [ ]:
# Process results list
for result in results[:10]:
    boxes = result.boxes  # Boxes object for bounding box outputs
    print(boxes.data.tolist()) # x1, y1, x2, y2, conf score, class id
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    # result.save(filename="result.jpg")  # save to disk

### From predict folder

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob("runs/detect/predict/*.jpg")[:10]:
  display(Image(filename=image_path, height=400))
  print("\n")

# 7. Run Model

## 7.1 Initalize DeepSORT

In [ ]:
from deep_sort_pytorch.deepsort import DeepSortTracker
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.utils.draw import draw_boxes
from deep_sort_pytorch.utils.tools import generate_idx_to_class

# Import idx_to_class file
if os.path.isfile(idx_to_class_path):
  with open(idx_to_class_path, 'r') as f:
    idx_to_class = json.load(f)
else:
  idx_to_class = generate_idx_to_class(dataset_data_path)

cfg = get_config() # Create new empty dict
cfg.merge_from_dict(deep_sort_cfg) # merge dict

# Initialize DeepSORT tracker
deep_sort_tracker = DeepSortTracker(cfg, use_cuda=True)

## 7.2 Images

In [ ]:
# Loop through images/frames
for img_path in glob.glob(dataset_path + "/valid/images/*.jpg"):

    frame = cv2.imread(img_path)
    if frame is None:
        continue

    # Perform object detection
    results = model.predict(source=frame, verbose=True)

    # Prepare detections for DeepSORT
    bbox_xywh = results[0].boxes.xywh.cpu().numpy()
    conf = results[0].boxes.conf.cpu().numpy()
    cls_ids = results[0].boxes.cls.cpu().numpy()

    # Update tracker
    outputs, _ = deep_sort_tracker.update(bbox_xywh, conf, cls_ids, frame) #outputs bbox, track id, class

    # Draw results
    if len(outputs) > 0:
      bbox_xyxy = outputs[:, :4] # Bounding Boxes
      identities = outputs[:, -1] # Unique Track IDs
      cls = outputs[:, -2] # Classes

      # Get class names
      names = [idx_to_class[str(label)] for label in cls]
      # Draw bounding boxes
      ori_im = draw_boxes(frame, bbox_xyxy, names, identities)
      # Display the frame
      cv2_imshow(ori_im)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

## 7.3 Video

In [ ]:
save_video = True
play_frames = False

cap = cv2.VideoCapture("/content/people.mp4") # Load video

ret, frame = cap.read() # Read first frame

# Retrieve frame dimensions and frames per second (fps)
if save_video:
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter
if save_video:
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
  out = cv2.VideoWriter('/content/output.mp4', fourcc, fps, (frame_width, frame_height))

while ret:
  # Perform object detection
  results = model.predict(source=frame, verbose=True)

  # Prepare detections for DeepSORT
  bbox_xywh = results[0].boxes.xywh.cpu().numpy()
  conf = results[0].boxes.conf.cpu().numpy()
  cls_ids = results[0].boxes.cls.cpu().numpy()

  # Update tracker
  outputs, _ = deep_sort_tracker.update(bbox_xywh, conf, cls_ids, frame) #outputs bbox, track id, class

  # Draw results
  if len(outputs) > 0:
    bbox_xyxy = outputs[:, :4] # Bounding Boxes
    identities = outputs[:, -1] # Unique Track IDs
    cls = outputs[:, -2] # Classes

    # Get class names
    names = [idx_to_class[str(label)] for label in cls]
    # Draw bounding boxes
    ori_im = draw_boxes(frame, bbox_xyxy, names, identities)

  if save_video: out.write(ori_im) # Write current frame to output
  if play_frames: cv2_imshow(ori_im) # output current frame

  ret, frame = cap.read() # Read next frame

  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
if save_video: out.release()
cv2.destroyAllWindows()

### Play Video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = "/content/output.mp4"
compressed_video_path = "/content/output_compressed.mp4"

# compress video file
os.system(f"ffmpeg -i {video_path} -vcodec libx264 {compressed_video_path}")

mp4 = open(compressed_video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
  <video width=640 controls>
      <source src="{data_url}" type="video/mp4">
  </video>
""")